In [12]:
import tensorflow as tf

In [18]:
class VQANet:
    def __init__(self, combine_type, question_embed_dim, lstm_dim, n_answers):
        self.combine_type = combine_type
        self.question_embed_dim = question_embed_dim
        self.lstm_dim = lstm_dim
        self.n_answers = n_answers
        self.build()
        
    def build(self):
        if self.combine_type == 'show-and-tell':
            dummy = tf.keras.layers.Input(input_shape=(20,), 
                                          dtype='float32')
            
            image_embedding = tf.keras.layers.Dense(units=self.question_embed_dim, 
                                              activation='elu', 
                                              name='image_embedding')(inputs=dummy)
            
            question_input = tf.keras.layers.Input(input_shape=(MAX_QUESTION_LEN,), 
                                                   dtype='int32',
                                                   name='question_input')
            
            question_embedding = tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, 
                                                           output_dim=self.question_embed_dim, 
                                                           mask_zero=True, 
                                                           input_length=MAX_QUESTION_LEN,
                                                           name='question_embedding')(inputs=question_input)
            
            image_question_embedding = tf.keras.layers.Concatenate(axis=1, 
                                                                   name='image_question_embedding')(inputs=[image_embedding, question_embedding])
            
            question_features, last_h, _ = tf.keras.layers.LSTM(units=self.lstm_dim, 
                                                                return_sequences=True, 
                                                                return_state=True, 
                                                                name='question_generator')(inputs=image_question_embedding)

            question_pred = tf.keras.layers.TimeDistributed(layer=Dense(units=VOCAB_SIZE, 
                                                                  activation='softmax', 
                                                                  name='word_classifier'))(inputs=question_features)
            
            answer_pred = tf.keras.layers.Dense(units=n_answers,
                                                activation='softmax'
                                                name='answer_classifier')(inputs=last_h)
            
            
            model = tf.keras.Model(inputs=[dummy, question_input], outputs=[question_pred, answer_pred])

            
            
            
            
            

SyntaxError: invalid syntax (<ipython-input-18-50a7b54d423d>, line 3)

In [17]:
class CNN:
    def __init__(self, path=None, type='vgg'):
        

INFO:tensorflow:Restoring parameters from ./weights/oreilly/model-500
[]
